In [1]:
from datetime import datetime
import os
import time
import requests
import json

import pickle
from pathlib import Path
import traceback

In [27]:
# call API to retrieve comments of a game with a game id

# game_steamid = 1716740          # starfield
# game_name = 'starfield'

# game_steamid = 2138330          # cyberpunk2077 phantom liberty
# game_name = 'cyberpunk2077_phantom_liberty'

# game_steamid = 1091500          # cyberpunk2077
# game_name = 'cyberpunk2077'

# game_steamid = 1118010
# game_name = 'monster_hunter_world_iceborne'

# game_steamid = 582010
# game_name = 'monster_hunter_world'

# game_steamid = 730
# game_name = 'counter-strike_2'

# game_steamid = 570
# game_name = "dota2"

# game_steamid = 1814460
# game_name = "the_brave_vs_dragon"

# game_steamid = 1814480
# game_name = 'fourplay_chess'

# game_steamid = 1607720
# game_name = 'rimebeard'

# game_steamid = 1814510
# game_name = 'yavi'

# game_steamid = 1814520
# game_name = 'find_the_cat'

# game_steamid = 1814540
# game_name = 'cosmos_conquer'

# game_steamid = 1757970
# game_name = 'oasis_dark_forest'

game_steamid = 1814560
game_name = 'the_infecting_3'

params = {
    'json':1,
    'cursor':'*',
    'num_per_page':100,
    'language':'english',       # only english reviews
    'filter':'recent',
    'review_type':'all',
    'purchase_type':'all'

}

# api doc: https://partner.steamgames.com/doc/store/getreviews
# reviews_req = requests.get(f"https://store.steampowered.com/appreviews/{game_steamid}", params=params)

In [28]:
# loop to get comments in range of a day

datetime_until = datetime(2010, 1, 1, 0, 0, 0)      # get until...

checkpoint_counter = 0
reviews_reqs = []

# get existing folder and retrieve the cursor object (?)

# load the latest file
game_folder = Path(f'./{game_name}')
if game_folder.exists():
    try:
        latest_file_path = sorted(game_folder.glob('steam_reviews_*.pkl'))[-1]
        with open(latest_file_path, 'rb') as f:
            reviews_reqs = pickle.load(f)           # retrieve the list of reviews
            cursor_str = reviews_reqs[-1]['cursor']
            print('Loaded:', latest_file_path)
    except IndexError as e:
        print('Error loading the latest file:', e)
        traceback.print_exc()
else:
    cursor_str = '*'

print('Cursor:', cursor_str)


save_folder = game_folder
if not save_folder.exists():
    os.makedirs(save_folder)

while True:
    params['cursor'] = cursor_str
    reviews_req = requests.get(f"https://store.steampowered.com/appreviews/{game_steamid}", params=params)

    reviews_json = reviews_req.json()
    if 'reviews' not in reviews_json:
        print('No reviews found. To the end of all reviews')
        break
    reviews = reviews_json['reviews']
    if len(reviews) == 0:
        print('No reviews found. To the end of all reviews')
        break
    reviews_reqs.append(reviews_json)
    cursor_str = reviews_json['cursor']
    print(f"Got {len(reviews)} reviews, cursor: {cursor_str}")
    
    if datetime.fromtimestamp(reviews[-1]['timestamp_created']) < datetime_until:
        print('Reached until date')
        break

    checkpoint_counter += 1
    if checkpoint_counter % 50 == 0:
        save_filepath = save_folder.joinpath(f'steam_reviews_{game_steamid}_{datetime.now().strftime("%Y%m%d%H%M%S")}.pkl')
        with open(save_filepath, 'wb') as f:
            pickle.dump(reviews_reqs, f)
        print('Saved to:', save_filepath)
        # time.sleep(5)

    time.sleep(1)

# save the reviews
with open(save_folder.joinpath(f'steam_reviews_{game_steamid}_{datetime.now().strftime("%Y%m%d%H%M%S")}.pkl'), 'wb') as f:
    pickle.dump(reviews_reqs, f)
print('Program terminated. Saved to:', save_folder.joinpath(f'steam_reviews_{game_steamid}_{datetime.now().strftime("%Y%m%d%H%M%S")}.pkl'))

Cursor: *
Got 8 reviews, cursor: AoJ4xKDbu/0Cd6zRkAM=
No reviews found. To the end of all reviews
Program terminated. Saved to: the_infecting_3/steam_reviews_1814560_20240311164921.pkl


In [16]:
len(reviews_reqs)

1

In [5]:
datetime.fromtimestamp(1707534074)      # its a unix timestamp, to our timezone (GMT+8).

datetime.datetime(2024, 2, 10, 11, 1, 14)

In [8]:
reviews_req.json()

{'success': 1,
 'query_summary': {'num_reviews': 0},
 'reviews': [],
 'cursor': 'AoJ4ooCl54oDeOnvsAQ='}

In [6]:
with open('reviews.json', 'w') as f:
    json.dump(reviews_req.json(), f, indent=2)

In [4]:
print(reviews_req.json()['reviews'][2]['review'])

Was initially disappointed with this game but its come along way and I seriously enjoy it. Cant wait for CK to be added since modding it right now is a massive pain
